## Part I : Classification

#### Imports:

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import multi_gpu_model

import keras 
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback

Using TensorFlow backend.


#### Models Parameters:

In [2]:
BATCH_SIZE = 16
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 1000000
MAX_TRAIN_EPOCHS = 5

In [3]:
# Use of tensorflow:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 17771304971034291601)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6664374190, 11012453760361821324)
Default GPU Device: /device:GPU:0


#### Paths to folders:

In [9]:
ship_dir = '../../data/airbus_ship_detection/'
ship_dir_thibault = '../../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir_thibault, 'train')# Images for training
test_image_dir = os.path.join(ship_dir_thibault, 'test')# Images for testing
model_weights_dir = "weights_models/"

label_dir = os.path.join(ship_dir_thibault, 'train_ship_segmentations_v2_clean.csv')# Images for testing
dataframe = pd.read_csv(label_dir, engine="python") # Markers for ships

In [10]:
data_link_balanced = balancing_train(dataframe, rate_of_has_ship=0.5, ship_dir_train=train_image_dir)
data_link_unbalanced = balancing_train(dataframe, rate_of_has_ship=0.0, ship_dir_train=train_image_dir)

print("data_link_balanced rate:{0}, lenght: {1}".format(data_link_balanced.has_ship.sum()/len(data_link_balanced)
                                                    ,len(data_link_balanced)))
print("data_link_unbalanced rate:{0}, lenght: {1}".format(round(data_link_unbalanced.has_ship.sum()/len(data_link_unbalanced),2)
                                                    ,len(data_link_unbalanced)))

data_link_balanced rate:0.5, lenght: 80432
data_link_unbalanced rate:0.21, lenght: 187097


In [11]:
batch_size = 8

In [13]:
train_gen = make_image_gen(data_link_unbalanced, train_image_dir, batch_size, (1, 1))
train_x, train_y = next(train_gen)

print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_x.min(), train_x.max())

# Definition of the model with the input shape
clf_model = get_resnet50_classif(input_shape=train_x.shape[1:])

# Load weights
weights_path = model_weights_dir + "model_clf_scal1_s_gpu_weights.best.h5"
print("Load weights...")
clf_model.load_weights(weights_path)

x (8, 768, 768, 3) 0.0 1.0
y (8,) 0.0 1.0
Load weights...


### Generate the new dataset for clustering

In [14]:
from keras.models import Sequential

new_model = Sequential()
for layer in clf_model.layers[:-1]:
    new_model.add(layer)

In [15]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
step_count = len(dataframe)//batch_size

In [ ]:
dataframe_gen = make_image_gen(data_link_unbalanced, train_image_dir, batch_size, (1, 1))
pred = new_model.predict_generator(dataframe_gen, steps=step_count, verbose=1)
pred.shape # (231722, 1)

 3430/28965 [==>...........................] - ETA: 2:40:08

In [28]:
pred[1]

array([0.00040627, 0.        , 0.00135422, ..., 0.00362741, 0.        ,
       0.01115465], dtype=float32)

In [ ]:
# pred.save(ship_dir_thibault + "GAP_layer_pred.npy")

In [ ]:
data_link_unbalanced["GAP_pred"] = pred

In [ ]:
data_link